In [3]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
# Added protobuf==3.20.3 to poetry, as I got same error as in https://github.com/autogluon/autogluon/issues/1725
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [22]:
host = "localhost:8500"

# Don' t need secure_channel with auth etc
# Because both Gateway and TF-Serving will run in
# Kubernetes and TF-Serving will not be accessible from the outside
channel = grpc.insecure_channel(host)
# prediction_service_pb2 is the TF-Serving component
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [23]:
from keras_image_helper import create_preprocessor

In [24]:
preprocessor = create_preprocessor("xception", target_size=(299, 299))

In [25]:
url = "http://bit.ly/mlbookcamp-pants"

X = preprocessor.from_url(url)

In [26]:
### Need to prepare the X for PB (protobuf) request
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [27]:
pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = "clothing-model"
pb_request.model_spec.signature_name = "serving_default"
pb_request.inputs["input_8"].CopyFrom(np_to_protobuf(X))

In [28]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [29]:
# To extract predictions
pb_response.outputs["dense_7"].float_val

[-1.8798645734786987, -4.756311893463135, -2.3595337867736816, -1.0892643928527832, 9.90378475189209, -2.826181411743164, -3.648310899734497, 3.241156816482544, -2.612095832824707, -4.852035999298096]

In [30]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(
    classes,
    pb_response.outputs["dense_7"].float_val
))

{'dress': -1.8798645734786987,
 'hat': -4.756311893463135,
 'longsleeve': -2.3595337867736816,
 'outwear': -1.0892643928527832,
 'pants': 9.90378475189209,
 'shirt': -2.826181411743164,
 'shoes': -3.648310899734497,
 'shorts': 3.241156816482544,
 'skirt': -2.612095832824707,
 't-shirt': -4.852035999298096}